# LangChain

## Demo code to use LangChain
A quick demo code to use LangChain.

Please ensure you have *.env* file in your HOME/Documents/src/openai/ folder

In [ ]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
from pprint import pprint
import os

env_path = os.getenv("HOME") + "/Documents/src/openai/.env"
load_dotenv(dotenv_path=env_path, verbose=True)

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://pvg-azure-openai-uk-south.openai.azure.com"

model = AzureChatOpenAI(deployment_name="gpt-35-turbo")

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
 
model = AzureChatOpenAI(deployment_name="gpt-35-turbo")
 
messages = [
    SystemMessage(content="你是中国古代的圣贤孔子。请以如下格式使用文言文回答问题 {'author': '','answer': ''}"),
    HumanMessage(content="我应该如何看待学习？"),
    AIMessage(content="{''author': '孔子',answer': '学而时习之，不亦说乎？'}"),
    HumanMessage(content="我应该如何过好这一生？")
]

print(model.invoke(messages).content)

In [ ]:
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("给一段《{book}》里的名言")
print(template)

print(model.invoke(template.format(book='论语')).content)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("你是中国古代的圣贤{person}。请以如下格式使用文言文回答问题 {{'author': '','answer': ''}}"),
        HumanMessagePromptTemplate.from_template("{query}")
    ]
)

print(template)

prompt = template.format_messages(person="孔子", query="我应该如何看待学习？")
print(model.invoke(prompt).content)

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser

class Answer(BaseModel):
    author: str = Field(..., description="Author")
    answer: str = Field(..., description="Answer")

    @validator("answer")
    def validate_answer(cls, field) -> str:
        return field

    @validator("author")
    def validate_answer(cls, field) -> str:
        return field

parser = PydanticOutputParser(pydantic_object=Answer)

template = """你是中国古代的圣贤{person}。请使用文言文回答问题。
{format_instructions}
用户输入:
{query}"""

prompt = PromptTemplate(
    template=template,
    input_variables=["person", "query"],
    partial_variables={"format_instructions": parser.get_format_instructions()} 
)

print(parser.get_format_instructions())

model_input = prompt.format_prompt(person="孔子",query="我应该如何看待学习？")
model_output= model.invoke(model_input.to_messages())

print(model_output.content)

In [ ]:
chain = prompt | model 
print(chain.invoke({"person":"孟子","query":"我应该如何看待学习？"}).content)

In [ ]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai.embeddings import AzureOpenAIEmbeddings

examples = [
    {"input": "2+3", "output": "5"},
    {"input": "我应该如何看待学习？", "output": "学而时习之，不亦说乎？"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = AzureOpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [ ]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=1,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "我应该如何过好这一生？"})

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

In [ ]:
print(few_shot_prompt.format(input="我应该如何过好这一生？"))

In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
print(few_shot_prompt.format(input="我应该如何过好这一生？"))

In [ ]:
chain = final_prompt | model

chain.invoke({"input": "我应该如何过好这一生？"})